In [1]:
from pathlib import Path
import sys
import sqlite3
import pandas as pd
import ast
from IPython.display import clear_output

sys.path.insert(0, str(Path.cwd().parent))
from rd.mol import smiles_to_xyz_block, beta_cleavage_indices, proton_transfer_indices
from util.py3 import from_xyz_block, visualize_traj
from util.sql import execute_append, refresh
from util.ref import METHOD_MAP
from util.mpl import visualize_scan_energy
from orca.writing import write_orca

refresh()

db = Path.home() / "C5O-Kinetics/db/data.db"
conn = sqlite3.connect(db)

smiles_text = str(input("Enter a SMILES string: ")).upper()

df_smiles = pd.read_sql_query(
    "SELECT * FROM smiles WHERE smiles_text = ?", conn, params=(smiles_text,)
)
df_methods = pd.read_sql_query("SELECT * FROM methods", conn)


def prompt_methods(mask):
    methods_dict = {
        int(
            row.method_id
        ): f"  {row.method_id:<10} | {row.method:<15} | {row.functional} {row.basis}"
        for row in df_methods[mask].itertuples()
    }

    print("\n  Select an Available Method: ")
    print("  " + "=" * 55)
    for val in methods_dict.values():
        print(val)

    while True:
        sel_id = int(input("\nSelect a method id: "))
        if sel_id in list(methods_dict.keys()):
            return df_methods[df_methods.method_id == sel_id]


if df_smiles.empty:
    while True:
        clear_output()
        multiplicity = str(input("Indicate the multiplicity: "))
        initial_xyz = smiles_to_xyz_block(smiles_text)
        from_xyz_block(initial_xyz)

        (proceed := True) if (
            input(
                f"Type (Y)es or (N)o to continue with shown structure & multiplicity {multiplicity}."
            ).lower()[0]
            == "y"
        ) else (proceed := False)

        if proceed:
            break

    smiles_id = execute_append(
        "INSERT INTO smiles (smiles_text, multiplicity, initial) VALUES (?, ?, ?)",
        (smiles_text, multiplicity, initial_xyz),
        db,
        return_id=True,
    )

    sel_method = prompt_methods(df_methods.method == "GOAT")
    shell_cmd = write_orca(
        smiles_text, int(sel_method.iloc[0]["method_id"]), initial_xyz
    )

else:
    smiles_id = df_smiles["smiles_id"].iloc[0]

    df_calculations = pd.read_sql_query(
        f"SELECT * FROM calculations WHERE smiles_id = {smiles_id}", conn
    )

    clear_output()
    print("\n  Select A Method Output To Pull Starting Structure")
    sel_method = prompt_methods(df_methods.method_id.isin(df_calculations.method_id))

    clear_output()
    calcs_dict = {
        row.calc_id: f"  {row.calc_id:<10} | {row.scan_idx1:<2} to {row.scan_idx2:<2}"
        for row in df_calculations[
            df_calculations.method_id == int(sel_method.iloc[0]["method_id"])
        ].itertuples()
    }

    if len(calcs_dict) > 1:
        print(
            "\n  Multiple Matching Calculations. Select A Calculation To Use As Starting Structure"
        )
        print("  " + "=" * 55)
        for val in calcs_dict.values():
            print(val)
        while True:
            calc_id = int(input("\nSelect a calculation id: "))
            if calc_id in list(calcs_dict.keys()):
                break
    else:
        calc_id = list(calcs_dict.keys())[0]

    clear_output()
    df_xyzs = pd.read_sql_query(
        "SELECT * FROM xyz WHERE calc_id = ?", conn, params=(calc_id,)
    )
    initial_xyz = df_xyzs.iloc[0]["xyz_text"]
    print("Selected step from trajectory: ", df_xyzs.iloc[0]["selected_step"])
    from_xyz_block(initial_xyz)

    if "OPTTS" in sel_method.iloc[0]["method"]:
        df_imag = pd.read_sql_query(
            "SELECT * FROM imaginaryfrequencies WHERE calc_id = ?", conn, params=(calc_id,)
        )
        imag_freq = df_imag.iloc[0]["xyz"]
        visualize_traj(imag_freq)

    if "SCAN" in sel_method.iloc[0]["method"]:
        df_traj = pd.read_sql_query(
            "SELECT * FROM traj WHERE calc_id = ?", conn, params=(calc_id,)
        )
        traj = df_traj.iloc[0]["traj_text"]
        visualize_traj(traj)
        df_energy = pd.read_sql_query(
            "SELECT * FROM energies WHERE calc_id = ?", conn, params=(calc_id,)
        )
        arr = ast.literal_eval(df_energy.iloc[0]["energy_array"])
        steps, energies = arr
        visualize_scan_energy(steps, energies)

    (proceed := True) if (
        input("Type (Y)es or (N)o to continue with shown structure(s).").lower()[0] == "y"
    ) else (proceed := False)

    if not proceed:
        raise KeyboardInterrupt

    idx1, idx2 = df_calculations[df_calculations.calc_id == calc_id].iloc[0, 3:]

    # Determine allowed next methods
    allowed_methods = METHOD_MAP.get(sel_method.iloc[0]["method"], [])
    sel_method = prompt_methods(df_methods.method.isin(allowed_methods))

    if "SCAN" in sel_method.iloc[0]["method"]:
        print("\n  Available Mechanisms")
        print("  " + "=" * 55)
        print("  1 | Proton Transfer")
        print("  2 | Beta Cleavage")
        while True:
            mechanism_inp = int(input("\nSelect a mechanism: "))
            if mechanism_inp == 1:
                mechanism = "proton transfer"
                idxs1, idxs2 = proton_transfer_indices(smiles_text)
                label1, label2 = "proton", "radical"
                break

            elif mechanism_inp == 2:
                mechanism = "beta cleavage"
                idxs1, idxs2 = beta_cleavage_indices(smiles_text)
                label1, label2 = "alpha", "beta"
                break

        while True:
            idx1 = int(input(f"\nSelect a {label1} index for {mechanism} {idxs1}: "))
            if idx1 in idxs1:
                break

        while True:
            idx2 = int(input(f"\nSelect a {label2} index for {mechanism} {idxs2}: "))
            if idx2 in idxs2:
                break

    else:
        idx1, idx2, mechanism = -1, -1, None

    shell_cmd = write_orca(
        smiles_text,
        int(sel_method.iloc[0]["method_id"]),
        initial_xyz,
        idx1,
        idx2,
        mechanism,
    )

clear_output()
print("\nCopy and paste the following command into your terminal:\n")
print(shell_cmd, "\n")



Copy and paste the following command into your terminal:

cd /home/tns97255/C5O-Kinetics/calc/154; sbatch submit.sh 

